In [3]:
# pip install scikit-learn
import sklearn 
import numpy as np
# from sagemaker import get_execution_role
# import sagemaker
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
import datetime
import time
import tarfile
import boto3
import pandas as pd

# sm_boto3 = boto3.client("sagemaker")
# sess = sagemaker.Session()
# _region = sess.boto_session.region_name
# bucket = 'sagemaker-tutorial-s3-bucket-test' # Mention the created S3 bucket name here
# print("Using bucket " + bucket)

In [4]:
df = pd.read_csv("OULAD_cleaned_data.csv")

In [7]:
# display how many rows and columns of the dataset
df.shape

(25843, 18)

In [21]:
df

,id_student,code_module,code_presentation,total_score*weight,attempted_weight,gender,highest_education,age_band,num_of_prev_attempts,studied_credits,disability,final_result
0,6516,1,4,6350.0,100.0,1,3,3,0,60,0,2
1,8462,4,2,3490.0,40.0,1,3,3,0,90,0,0
2,8462,4,4,4300.0,50.0,1,3,3,1,60,0,0
3,11391,1,2,8240.0,100.0,1,3,3,0,240,0,2
4,23629,2,1,1669.0,25.0,0,1,1,2,60,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
25838,2698257,1,2,6940.0,100.0,1,1,1,0,120,0,2
25839,2698535,3,3,937.0,39.0,1,1,1,0,60,0,0
25840,2698535,5,2,5344.0,100.0,1,1,1,0,60,0,2
25841,2698577,2,4,5580.0,100.0,0,1,2,0,60,0,1


In [8]:
# show all rows of highest_education=="HE Qualification"
HE_num = len(df[df.highest_education=="HE Qualification"])
LowerA = len(df[df.highest_education=="Lower Than A Level"])
LevelA= len(df[df.highest_education=="A Level or Equivalent"])
PG_num=len(df[df.highest_education=="Post Graduate Qualification"])
NoForm=len(df[df.highest_education=="No Formal quals"])

print(HE_num)
print(LowerA)
print(LevelA)
print(PG_num)
print(NoForm)

3990
9847
11496
280
230


In [22]:
# preprocess "highest_education" column, Lower Than A Level:1,A Level or Equivalent:2,HE Qualification:3, postgraduate:4,no formal quals:0
change={"No Formal quals":0, "Lower Than A Level":1, "A Level or Equivalent":2, "HE Qualification":3, "Post Graduate Qualification":4}
df["highest_education"] = df["highest_education"] .map(change)

# change code_module 
df["code_module"] = df["code_module"].map({"AAA":1, "BBB":2, "CCC":3, "DDD":4, "EEE":5, "FFF":6, "GGG":7})

# change code_presentation
df["code_presentation"] = df["code_presentation"].map({"2013B":1, "2013J":2, "2014B":3, "2014J":4})

# change gender
df["gender"] = df["gender"].map({"M":1, "F":0})

# change age_band
df["age_band"] = df["age_band"].map({"0-35":1, "35-55":2, "55<=":3})

#change disability
df["disability"] = df["disability"].map({"N":0, "Y":1})


In [10]:
# delete region and imd_band columns
newdf = df.drop(columns=["region", "imd_band"])

In [11]:
# change final_result
newdf["final_result"] = newdf["final_result"].map({"Withdrawn":0,  "Fail":1,  "Pass":2, "Distinction":3})
# delete module_presentation column
df=newdf.drop(columns=["module_presentation"])

In [12]:
# delete adjusted_mark and mark
df = df.drop(columns=["adjusted_mark","mark"])

In [13]:
# delete Unnamed column
df = df.drop(columns=["Unnamed: 0"])

In [14]:
display(df)

,id_student,code_module,code_presentation,total_score*weight,attempted_weight,gender,highest_education,age_band,num_of_prev_attempts,studied_credits,disability,final_result
0,6516,1,4,6350.0,100.0,1,3,3,0,60,0,2
1,8462,4,2,3490.0,40.0,1,3,3,0,90,0,0
2,8462,4,4,4300.0,50.0,1,3,3,1,60,0,0
3,11391,1,2,8240.0,100.0,1,3,3,0,240,0,2
4,23629,2,1,1669.0,25.0,0,1,1,2,60,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
25838,2698257,1,2,6940.0,100.0,1,1,1,0,120,0,2
25839,2698535,3,3,937.0,39.0,1,1,1,0,60,0,0
25840,2698535,5,2,5344.0,100.0,1,1,1,0,60,0,2
25841,2698577,2,4,5580.0,100.0,0,1,2,0,60,0,1


In [15]:
# dividing data to train data and test data
train_data, test_data = np.split(df.sample(frac=1, random_state=1729), [int(0.7 * len(df))])
print(train_data.shape, test_data.shape)

(18090, 12) (7753, 12)


In [16]:
# write to csv file with combined tables
pd.DataFrame(train_data).to_csv('oulad_train.csv')
pd.DataFrame(test_data).to_csv('oulad_test.csv')

In [3]:
%%writefile oulad_script.py
# create training script (top line has to be the first line in a cell)

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, precision_score, recall_score, f1_score, roc_curve, auc
import sklearn
import joblib
import boto3
import pathlib
from io import StringIO 
import argparse
import joblib
import os
import numpy as np
import pandas as pd

# inference functions ---------------

# def input_fn(request_body, request_content_type):
#     print(request_body)
#     print(request_content_type)
#     if request_content_type == "text/csv":
#         request_body = request_body.strip()
#         try:
#             df = pd.read_csv(StringIO(request_body), header=None)
#             return df
        
#         except Exception as e:
#             print(e)
#     else:
#         return """Please use Content-Type = 'text/csv' and, send the request!!""" 
 
    
def model_fn(model_dir):
    clf = joblib.load(os.path.join(model_dir, "model.joblib"))
    return clf

# def predict_fn(input_data, model):
#     if type(input_data) != str:
#         prediction = model.predict(input_data)
#         print(prediction)
#         return prediction
#     else:
#         return input_data
        
    
if __name__ == "__main__":

    print("[INFO] Extracting arguments")
    parser = argparse.ArgumentParser()

    # hyperparameters sent by the client are passed as command-line arguments to the script.
    parser.add_argument("--n_estimators", type=int, default=100)
    parser.add_argument("--random_state", type=int, default=0)

    # Data, model, and output directories
    parser.add_argument("--model-dir", type=str, default=os.environ.get("SM_MODEL_DIR"))
    parser.add_argument("--train", type=str, default=os.environ.get("SM_CHANNEL_TRAIN"))
    parser.add_argument("--test", type=str, default=os.environ.get("SM_CHANNEL_TEST"))
    parser.add_argument("--train-file", type=str, default="oulad_train.csv")
    parser.add_argument("--test-file", type=str, default="oulad_test.csv")

    args, _ = parser.parse_known_args()
    
    print("SKLearn Version: ", sklearn.__version__)
    print("Joblib Version: ", joblib.__version__)

    print("[INFO] Reading data")
    print()
   
    train_df = pd.read_csv(os.path.join(args.train, args.train_file))
    test_df = pd.read_csv(os.path.join(args.test, args.test_file))
    
    features = list(train_df.columns)
    label = "final_result"
    
    print("Building training and testing datasets")
    print()
    
    # change  "total_score*weight"  because target column has to be numerical
    
    X_train = train_df.drop( label, axis=1)
    y_train = train_df[label]
    X_test = test_df.drop(label, axis=1)
    y_test = test_df[label]

    print('Column order: ')
    print(features)
    print()
    
    print("Label column is: ",label)
    print()
    
    print("Data Shape: ")
    print()
    print("---- SHAPE OF TRAINING DATA (70%) ----")
    print(X_train.shape)
    print(y_train.shape)
    print()
    print("---- SHAPE OF TESTING DATA (30%) ----")
    print(X_test.shape)
    print(y_test.shape)
    print()
    
  
    print("Training RandomForest Model.....")
    print()
    model =  RandomForestClassifier(n_estimators=args.n_estimators, random_state=args.random_state, verbose = 3,n_jobs=-1)
    # print("model is --------")
    # print(model)
    # print("X_train, y_train:----------")
    # print(X_train)
    # print(y_train)
    model.fit(X_train, y_train)
    print()
    

    model_path = os.path.join(args.model_dir, "model.joblib")
    joblib.dump(model,model_path)
    print("Model persisted at " + model_path)
    print()

    
    y_pred_test = model.predict(X_test)
    test_acc = accuracy_score(y_test,y_pred_test)
    test_rep = classification_report(y_test,y_pred_test)

    print()
    print("---- METRICS RESULTS FOR TESTING DATA ----")
    print()
    print("Total Rows are: ", X_test.shape[0])
    print('[TESTING] Model Accuracy is: ', test_acc)
    print('[TESTING] Testing Report: ')
    print(test_rep)
    
    # Tree Visualisation
from sklearn.tree import export_graphviz
import graphviz
# from sklearn import tree
# import matplotlib.pyplot as plt

# Export the first three decision trees from the forest
rf = model
cn=['Withdrawn', 'Fail', 'Pass', 'Distinction']
for i in range(3):
    tree = rf.estimators_[i]
    dot_data = export_graphviz(tree,
                               feature_names=X_train.columns,  
                               class_names=cn,
                               filled=True,  
                               max_depth=2, 
                               impurity=False, 
                               proportion=True)
    graph = graphviz.Source(dot_data)
    print("******graph******")
    print(graph)
    # Image(data=graph)
    # display(graph) # can't use it

Overwriting oulad_script.py


In [4]:
! python3 oulad_script.py --n_estimators 100 \
                   --random_state 0 \
                   --model-dir ./ \
                   --train ./ \
                   --test ./ \

[INFO] Extracting arguments
SKLearn Version:  1.2.2
Joblib Version:  1.2.0
[INFO] Reading data

Building training and testing datasets

Column order: 
['Unnamed: 0', 'id_student', 'code_module', 'code_presentation', 'total_score*weight', 'attempted_weight', 'gender', 'highest_education', 'age_band', 'num_of_prev_attempts', 'studied_credits', 'disability', 'final_result']

Label column is:  final_result

Data Shape: 

---- SHAPE OF TRAINING DATA (70%) ----
(18090, 12)
(18090,)

---- SHAPE OF TESTING DATA (30%) ----
(7753, 12)
(7753,)

Training RandomForest Model.....

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.
building tree 1 of 100building tree 2 of 100

building tree 3 of 100
building tree 4 of 100
building tree 5 of 100
building tree 6 of 100building tree 7 of 100
building tree 8 of 100

building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100building tree 14 of 100

building tree 15 o

In [19]:
# pip install graphviz

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.0/47.0 kB 3.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.
